# Importações

In [37]:
from langchain_text_splitters import RecursiveCharacterTextSplitter # pip install langchain-text-splitters

from sentence_transformers import SentenceTransformer # pip install sentence_transformers

import pdfplumber # pip install pdfplumber 

 
# Para tratamento de texto 

import re 

import spacy # python -m spacy download pt_core_news_sm 

import nltk # pip install nltk 

from nltk.corpus import stopwords 

# Download das stop words

nltk.download('stopwords')

import chromadb 

from langchain_community.vectorstores import Chroma 

[nltk_data] Downloading package stopwords to C:\Users\Felipe
[nltk_data]     Pereira\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
import chromadb
print("chromadb funcionando!")


chromadb funcionando!


# PLN (Processamento de linguagem natural)

* Leitura do PDF e contagem de caracteres e linhas do texto.


In [39]:
import pdfplumber

# Função para ler o pdf
def ler_pdf(caminho_pdf):
    leitor_pdf = pdfplumber.open(caminho_pdf)  # abre o pdf
    texto = ""  # string vazia para guardar o texto

    # passa por todas as páginas do pdf
    for pagina in range(len(leitor_pdf.pages)):
        pagina_texto = leitor_pdf.pages[pagina].extract_text()
        if pagina_texto:
            texto += pagina_texto + "\n"   # mantemos a quebra de linha

    # contar quantas linhas tem antes de remover \n
    numero_linhas = len(texto.splitlines())

    # agora removemos quebras de linha, se necessário
    texto = texto.replace("\n", " ")

    return texto, numero_linhas


# Carregar os documentos do PDF
arquivo_pdf = r"C:\Users\Felipe Pereira\Downloads\comportamento.pdf"
texto_pdf, total_linha = ler_pdf(arquivo_pdf)  # chama a função e armazena o texto

# Tamanho do texto
print("Tamanho do texto em caracteres:", len(texto_pdf))

# Número de linhas do PDF
print("Número total de linhas:", total_linha)

# Arquivo PDF original
print(texto_pdf)


Tamanho do texto em caracteres: 929
Número total de linhas: 13
Os cientistas estavam analisando grandes volumes de dados para descobrir padrões que pudessem prever o comportamento humano. Embora os resultados iniciais fossem promissores, ainda havia muito a ser feito. Alguns algoritmos aprenderam rapidamente, enquanto outros exigiram ajustes finos e reprocessamento de informações. A equipe percebeu que o contexto desempenhava um papel crucial na interpretação dos dados coletados. Durante os testes, surgiram desafios inesperados relacionados à ambiguidade linguística e à qualidade das fontes utilizadas. Mesmo frases simples podiam assumir significados diferentes dependendo do domínio em que eram aplicadas. Ao final da primeira fase do projeto, os pesquisadores estavam confiantes de que as técnicas de inteligência artificial estavam evoluindo, mas também conscientes das limitações atuais. Era necessário combinar abordagens estatísticas com conhecimento linguístico mais profundo. 


# Questão 2.B

O tamanho do texto em caracteres é de 929
O número de linhas que contem no texto é 13

# Questão 2.C

Como estou usando tamanho pequeno de 200 caracteres para o chunk, meu objetivo é aumentar o número de palavras chaves em um chunk, se eu deixasse as stopwords elas ocupariam um espaço grande sendo que são palavras de baixo valor semântico.
Escolhas das minhas stopwords: estou buscando remover artigos frequentes que aparecem no texto e vogais como u e i que não tem muito valor semântico. Assim busco eliminar o máximo de ruído que posso.

# Questão 2.D

Transformo o texto todo em mínusculo pois a letra maíuscula é um ruído. Exemplo disso: se eu tiver em um mesmo documento as palavras Erro e erro, elas viraram dois vetores diferentes, so que essa palavra tem o mesmo valor semântico então a transformação do texto para minusculo é extremamente importante para a normalização, visto que não sobe para o VectorDB vetores com o mesmo significado.

# Questão 2.E

Preferi usar a lemmatização do que a stemização porque por meio da lemmatização eu consigo manter o significado semântico da palavra. Além disso, busco manter o significado porque meu documento é sobre o assunto de Inteligência Artificial e cada palavra é importante para o contexto e entendimento do texto. Demonstro isso com exemplos abaixo:

    Stematização: rapidez importa
        ["correndo", "correção"] -> ["corr"]
        Perde o significado (é bastante usado para analise de documentos com infos rapidas, calculos)
        A partir do exemplo, consigo analisar que o modelo não saberia diferenciar correndo e correção.

    Lemmatização: contexto do texto importa
        ["correndo", "correção"] -> ["correr", "correção"] sei o significado
        Ela transforma substantivos para o singular, verbos para sua forma no infinitivo e etc.
Gerar

Optei por deixar os acentos e os números, pois a remoção dos acentos pode perder significado da palavra, visto que a língua portuguesa utiliza muito. Com a remoção do acento, posso mudar completamente o conceito que o vetor representa. Também vi que no meu documento os números são importantes pois referenciam datas (anos). Além disso estou usando um modelo moderno de embedding (model = Sentence Transformer('all-MiniLM-L6-v2')), o qual já é capaz de entender essa letras e acentuações do português.

Na parte de tokenização com spacy, uso a tokenização por palavra. Isso é uma prática de extrema importância, pois para eu saber se uma palavra é stopword ou não eu não posso comparar com um texto inteiro, tem que ser por palavras. Ademais, faço a lemmatização também, que consiste em transformar uma palavra na sua forma mais primitiva que existe (ex: correndo -> correr), por isso da tokenização por palavra.

    - Essa etapa de pré-processamento é diferente da tokenização por sub-palavra, que o próprio modelo de embedding usa internamente para gerar os vetores.

    

In [40]:
import spacy
import re
from nltk.corpus import stopwords

# Carregar o modelo de linguagem do spacy
nlp = spacy.load("pt_core_news_sm")

# definir stopwords
api_stop_words = set(stopwords.words('portuguese'))  # carrega as stopwords padrões
minhas_stop_words = {'a', 'e', 'i', 'o', 'u'}        # stopwords personalizadas
stop_words = api_stop_words | minhas_stop_words      # une as stopwords

# Função para fazer o tratamento de linguagem natural usando spacy
def tratamento_pln(texto):
    # normalização: tranforma o texto todo em minúsculo
    texto = texto.lower()

    # mantém letras, acentos E NÚMEROS, mas remove pontuação
    texto = re.sub(r'[^a-zA-Z0-9áéíóúâêôãõç\s]', '', texto)

    # tokenização com spacy
    doc = nlp(texto)

    # remoção de stopwords, pontuação e lematização
    clean_tokens = [
        token.lemma_ 
        for token in doc 
        if token.text not in stop_words and not token.is_punct
    ]

    # juntar tokens lematizados de volta em uma string
    clean_text = " ".join(clean_tokens)

    return clean_text


# Visualizando as Stop Words
print("Tamanho do conjunto stop_words:", len(stop_words))
print("\nStop_words ordenadas:\n", sorted(list(stop_words)))

# Chamada de PLN
texto_pdf_tratado = tratamento_pln(texto_pdf)

# Tamanho do texto original
print("Tamanho do texto original em caracteres:", len(texto_pdf))

# Arquivo PDF tratado
print("\nTEXTO TRATADO:\n")
print(texto_pdf_tratado)

# Tamanho do texto tratado
print("\nTamanho do texto tratado em caracteres:", len(texto_pdf_tratado))


Tamanho do conjunto stop_words: 209

Stop_words ordenadas:
 ['a', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'essa', 'essas', 'esse', 'esses', 'esta', 'estamos', 'estar', 'estas', 'estava', 'estavam', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéramos', 'estivéssemos', 'estou', 'está', 'estávamos', 'estão', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'forem', 'formos', 'fosse', 'fossem', 'fui', 'fôramos', 'fôssemos', 'haja', 'hajam', 'hajamos', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houveram', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houvermos', 'houverá', 'houverão', 'houveríamos', 'houvesse', 'houvessem', 'h

# Chunking e Embedding

- O embedding é o que lida com a vetorização, pega os Ids Numéricos da tokenização e os transforma em vetores, juntando o significado + contexto e armazenam no VectorDB
    -Exemplo: o id 2 -> [0.2, 0.8, -0.1, ..., 0.5]

# Questão 3.A

    -Estou usando o RecursiveCharacter TextSplitter para quebrar o texto em chunks. O recursive chunk é um método recursivo que preserva ao máximo a estrutura semântica do texto, basicamente ele olha o tamanho do chunk que eu defino no código abaixo, se o chunk for maior que 200 ele tenta quebrar o texto usando o próximo separador que seria por linha, quando o chunk for menor que o tamanho definido o processo acaba. Escolhi o método recursivo buscando preservar o contexto do texto, assim não haverá tanta perca semântica.

    -O tamanho do chunk escolhido foi de 200 caracteres, escolhi esse tamanho porque não quero perder o contexto e também quero que o meu chunk não seja tão grande para que ele me retorne apenas a resposta consolidada. Fiz a escolha do overlap de 50 caracteres, sendo mais que 10% do chunk original para que não haja a perca do contexto.

# Questão 3.B

    -Optei por fazer embedding de frase, usando o Sentence Transformer. Novamente meu objetivo é manter o contexto do texto, fazendo a vetorização por frases completas consigo fazer a diferença de frases como "Fui ao banco da praça" e "Sentei no banco da praça". Se eu escolhesse o embedding por palavras (Word Embeddings) a palavra banco teriam o mesmo vetor (significado) sem o contexto da frase.    



In [41]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

# Preparação do Texto para BD Vetorial (chunks)

# Criando o text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,      # tamanho máximo de cada chunk
    chunk_overlap=50     # overlap para manter o contexto
)

# Aplica o chunk no texto limpo
chunks = text_splitter.split_text(texto_pdf_tratado)

print("Chunks gerados:\n", chunks)
print("\nQuantidade de chunks:", len(chunks))

# Criação do embedding (transforma chunks em vetores)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Converte lista de chunks em embeddings
embeddings = model.encode(chunks)

# Gerando IDs automaticamente: doc_0, doc_1, doc_2...
uids = [f"doc_{i}" for i in range(len(chunks))]

print("\nIDs gerados:", uids)
print("Quantidade de embeddings:", len(embeddings))


Chunks gerados:
 ['cientista analisar grande volume dado descobrir padrão poder prever comportamento humano embora resultado inicial promissor ainda haver fazer algum algoritmos aprender rapidamente enquanto outro', 'algoritmos aprender rapidamente enquanto outro exigir ajuste fino reprocessamento informação equipe perceber contexto desempenhar papel crucial interpretação dado coletar durante teste surgir desafio', 'dado coletar durante teste surgir desafio inesperado relacionar   ambiguidade linguístico   qualidade fonte utilizar frase simples poder assumir significado diferente depender domínio aplicar final', 'diferente depender domínio aplicar final primeiro fase projeto pesquisador confiante técnica inteligência artificial evoluir consciente limitação atual necessário combinar abordagem estatístico', 'atual necessário combinar abordagem estatístico conhecimento linguístico profundo']

Quantidade de chunks: 5

IDs gerados: ['doc_0', 'doc_1', 'doc_2', 'doc_3', 'doc_4']
Quantidade de

In [42]:
## Aplicação do ChromaDB

# Criar o banco de dados
client = chromadb.Client()

# Criar a coleção (tipo uma tabela)
collection = client.create_collection(name="machlrn")
# client.delete_collection("machlrn")  # comando para apagar a coleção (caso queira resetar)

# Adicionar os documentos ao banco de dados
collection.add(
    documents=chunks,        # lista de textos dos chunks
    embeddings=embeddings,   # lista de vetores
    ids=uids                 # lista de IDs
)

#print("Dados adicionados com sucesso ao ChromaDB!")


 - Anotações sobre como é armazenado no banco

    - Embeddings -> indíce de busca, meu banco pega minha query(pergunta) e compara com os vetores numéricos até achar o mais similar (só localiza)
    - Documents -> é uma lista de chunks de texto, o banco pega a resposta mais similar e fornece esse conteúdo de volta
    - Ids -> chave de ligação entre o embedding e documents

In [43]:
# Aqui estou fazendo um guardrail de entrada
# Consiste basicamente em analisar a pergunta do usuário e ver se ela tem relação com o documento.
# Além disso posso bloquear palavras de ódio ou assuntos que não quero permitir.

palavras_bloqueadas = ['comida', 'assassinato', 'futebol', 'política', 'morte']

def query_guardrail(query_text):
    """Verifica se a pergunta contém palavras bloqueadas."""
    
    query_lower = query_text.lower()  # transforma a frase para minúsculo
    
    for word in palavras_bloqueadas:  # percorre a lista de palavras bloqueadas
        if word in query_lower:       # se a palavra estiver na pergunta
            print(f"Pergunta inválida. O tópico '{word}' não tem relação com o documento.")
            return False
    
    # se estiver tudo certo, aprova a pergunta
    return True


In [45]:
# Realizar a busca usando collection.query

# Query 1
# query_text = "Quem foram os criados do GPS?"  
# Query 2
# query_text = "O que é agente?"
query_text = "Qual a porcao necessaria de comida?"  # teste do guardrail

# aplicando o guardrail de entrada
if query_guardrail(query_text):
    print("Pergunta aprovada. Iniciando busca...")

    # transformar pergunta em embedding
    query_embedding = model.encode([query_text])

    # consultar o vetor no banco ChromaDB
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=1
    )

    print(results)  # resultado bruto

else:
    print("Desculpe, só posso responder perguntas sobre o documento fornecido.")


Pergunta inválida. O tópico 'comida' não tem relação com o documento.
Desculpe, só posso responder perguntas sobre o documento fornecido.


In [46]:
# se tiver info sensível no documento, o guardrail de recuperação não permite acessar

palavras_sensiveis = ['confidencial', 'secreto', 'senha', 'salário']

def document_guardrail(document_text, doc_id="Documento"):
    """Verifica se o documento recuperado contém palavras sensíveis."""
    
    doc_lower = document_text.lower()

    for word in palavras_sensiveis:
        if word in doc_lower:
            print(f"O documento '{doc_id}' foi bloqueado por conter informações sensíveis ({word}).")
            return False
    
    # se estiver tudo OK
    return True


In [47]:
import numpy as np

# Função de similaridade do cosseno
def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Função de distância euclidiana
def euclidean_dist(a, b):
    return np.linalg.norm(a - b)

# Gerar embedding do chunk retornado na busca
chunk_result = model.encode([results['documents'][0][0]])


In [48]:
# Imprimir os resultados de forma organizada
# Fazendo a varredura sobre os campos 'ids', 'distances' e 'documents'

for i in range(len(results['ids'][0])):

    doc_id = results['ids'][0][i]
    distance = results['distances'][0][i]  # quanto menor, mais similar
    document = results['documents'][0][i]  # texto do chunk encontrado

    # Guardrail de recuperação
    if document_guardrail(document):

        print(f"ID: {doc_id}")
        print(f"Distância: {distance}")
        print(f"Documento: {document}")

        # Gerar embedding do chunk atual
        chunk_vector = model.encode([document])[0]

        # Similaridades
        print("Cosseno:", cosine_sim(query_embedding[0], chunk_vector))
        print("Euclidiana:", euclidean_dist(query_embedding[0], chunk_vector))

        print("-" * 40)


ID: doc_0
Distância: 1.2529549598693848
Documento: cientista analisar grande volume dado descobrir padrão poder prever comportamento humano embora resultado inicial promissor ainda haver fazer algum algoritmos aprender rapidamente enquanto outro
Cosseno: 0.37352252
Euclidiana: 1.1193547
----------------------------------------
